In [2]:
import pandas as pd
from datetime import date
import numpy as np
import openpyxl
import xlsxwriter

In [3]:
gwp_factors = pd.read_excel('data/GHG_GWP_Factors.xlsx', header=0)
activity_data_user = pd.read_excel('data/Activity_Data_Sales_User.xlsx', header=0)
activity_data_producer = pd.read_excel('data/Activity_Data_Sales_Producer.xlsx', header=0)

In [4]:
activity_data = pd.concat([activity_data_user, activity_data_producer], axis=0, ignore_index=True)
activity_data

,activity_id,date_month_year,refrigerant_type,refrigerant_amount_start_year,refrigerant_amount_end_year,refrigerant_obtained_by_purchased,refrigerant_obtained_by_manufactures,refrigerant_obtained_by_contractors,refrigerant_obtained_but_returned,refrigerant_sales_bulk,...,activity_type,description,country,refrigerant_obtained_by_purchase,refrigerant_returned_by_users,refrigerant_sales_charged_equipment,refrigerants_sales_delivered,refrigerant_sales_returned,refrigerant_recycled,refrigerant_sales_destruct
0,1,2022,HFC-134a,500,50,50.0,100.0,34.0,50.0,10.0,...,Sales_Approach_User,office,DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,2022,HFC-236fa,5000,500,424.0,454.0,66.0,45.0,56.0,...,Sales_Approach_User,office,DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2022,PFC-14,350,50,798.0,808.0,98.0,40.0,102.0,...,Sales_Approach_User,office,DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2022,HFE-245cb2,200,5,1172.0,1162.0,130.0,35.0,148.0,...,Sales_Approach_User,office,DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2022,HFE-356pcc3,600,253,1546.0,1516.0,162.0,30.0,194.0,...,Sales_Approach_User,office,DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,6,2022,HFE 263fb2,671,35,1920.0,1870.0,194.0,25.0,240.0,...,Sales_Approach_User,office,DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,7,2022,HFC-236ea,15000,2235,2294.0,2224.0,226.0,20.0,286.0,...,Sales_Approach_User,office,DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,8,2022,HFC-43-10mee,11465,252,2668.0,2578.0,258.0,15.0,332.0,...,Sales_Approach_User,office,DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,9,2022,HFC-143a,32345,235,3042.0,2932.0,290.0,10.0,378.0,...,Sales_Approach_User,office,DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10,2022,HFC-245fa,21423,2523,3416.0,3286.0,322.0,5.0,424.0,...,Sales_Approach_User,office,DE,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
gwp_factors

,id,industrial_name,chemical_formula,gwp_factor,regulation,period
0,1,Carbon dioxide,co2,1.0,SAR,100 year
1,2,Methane,ch4,21.0,SAR,100 year
2,3,Nitrous oxide,n2o,310.0,SAR,100 year
3,4,Carbon dioxide,co2,1.0,AR4,100 year
4,5,Methane,ch4,25.0,AR4,100 year
...,...,...,...,...,...,...
99,100,Methane,ch4,86.0,AR5,20 year
100,101,Nitrous oxide,n2o,268.0,AR5,20 year
101,102,Carbon dioxide,co2,1.0,AR6,20 year
102,103,Methane,ch4,82.5,AR6,20 year


In [5]:
output_fugitive_emissions = pd.DataFrame({
    'date_month_year': pd.Series(dtype='str'),
    'refigerant_type': pd.Series(dtype='str'),
    'amount_stored': pd.Series(dtype='int'),
    'unit': pd.Series(dtype='str'),
    'scope': pd.Series(dtype='str'),
    'category_details': pd.Series(dtype='str'),
    'emission_co2e': pd.Series(dtype='int'),
    'co2e_calculation_method': pd.Series(dtype='str'),
    'co2e_unit': pd.Series(dtype='str'),
    'description': pd.Series(dtype='str'),
    'country': pd.Series(dtype='str')
    })

In [6]:
def get_Column(row, colum:str):
    return row[colum]

def get_date(row):
    return get_Column(row=row, colum='date_month_year')

def get_refigerant_type(row):
    return get_Column(row=row, colum='refrigerant_type')

def get_amount(row):
    amount = row['refrigerant_amount_start_year'] - row['refrigerant_amount_end_year']
    if amount < 0:
        return -1
    return amount

def get_unit(row):
    return get_Column(row=row, colum='unit')

def get_scope(row):
    return 'scope1'

def get_gwp_factor(regulation, industrial_name):
    gwp_factor = gwp_factors.loc[(gwp_factors['industrial_name'] == industrial_name) & (gwp_factors['regulation'] == regulation)].iloc[0]
    return gwp_factor['gwp_factor']

def get_emssion_c02e(row):
    year = 2023
    gwp_regulation = 'AR5'
    if(row['activity_type']) == "Sales_Approach_User":
        gwp_factor_industrial = get_gwp_factor(gwp_regulation, row['refrigerant_type'])
        amount_stored = row['refrigerant_amount_start_year'] - row['refrigerant_amount_end_year']
        if amount_stored < 0:
            amount_stored = -1
        amount_obtained = row['refrigerant_obtained_by_purchased'] + row['refrigerant_obtained_by_manufactures'] + row['refrigerant_obtained_by_contractors'] + row['refrigerant_obtained_but_returned']
        if amount_obtained < 0:
            amount_obtained = -1
        amount_sales = row['refrigerant_sales_bulk'] + row['refrigerant_sales_leftover'] + row['refrigerant_sales_returned_supplier'] + row['refrigerant_sales_returned_recycler'] + row['refrigerant_sales_returned_for_destruction']
        if amount_sales < 0:
            amount_sales = -1
        amount_fullcharge = row['refrigerant_fullcharge_new'] + row['refrigerant_fullcharge_retrofitted'] - row['refrigerant_fullcharge_retired'] - row['refrigerant_fullcharge_replaced_with_other']
        if amount_fullcharge < 0:
            amount_fullcharge = -1
        amount_sum = amount_stored + amount_obtained - amount_sales - amount_fullcharge
        if amount_sum < 0:
            amount_sum = -1
        return amount_sum * gwp_factor_industrial
    if(row['activity_type']) == "Sales_Approach_Producer":
        gwp_factor_industrial = get_gwp_factor(gwp_regulation, row['refrigerant_type'])
        amount_stored = row['refrigerant_amount_start_year'] - row['refrigerant_amount_end_year']
        if amount_stored < 0:
            amount_stored = -1
        amount_obtained = row['refrigerant_obtained_by_purchase'] + row['refrigerant_returned_by_users'] + row['refrigerant_sales_charged_equipment']
        if amount_obtained < 0:
            amount_obtained = -1
        amount_sales = row['refrigerants_sales_delivered'] + row['refrigerant_sales_returned'] + row['refrigerant_recycled'] + row['refrigerant_sales_destruct']
        if amount_sales < 0:
            amount_sales = -1
        amount_sum = amount_stored + amount_obtained - amount_sales
        if amount_sum < 0:
            amount_sum = -1
        return amount_sum * gwp_factor_industrial
    return -1
        
def get_emission_unit(row):
    return 'kg'

def get_calculation_method(row):
    if row['activity_type'] == 'Sales_Approach_User':
        return 'sales_approach_user'
    if row['activity_type'] == 'Sales_Approach_Producer':
        return 'sales_approach_producer'
    return 'unknown'

def get_description(row):
    return get_Column(row=row, colum='description')

def get_country(row):
    return get_Column(row=row, colum='country')    

output_fugitive_emissions['date_month_year'] = activity_data.apply(get_date, axis=1)
output_fugitive_emissions['refigerant_type'] = activity_data.apply(get_refigerant_type, axis=1)
output_fugitive_emissions['amount_stored'] = activity_data.apply(get_amount, axis=1)
output_fugitive_emissions['unit'] = activity_data.apply(get_unit, axis=1)
output_fugitive_emissions['scope'] = activity_data.apply(get_scope, axis=1)
#output_fugitive_emissions['category_details'] = activity_data.apply(get_date, axis=1)
output_fugitive_emissions['emission_co2e'] = activity_data.apply(get_emssion_c02e, axis=1)
output_fugitive_emissions['co2e_calculation_method'] = activity_data.apply(get_calculation_method, axis=1)
output_fugitive_emissions['co2e_unit'] = activity_data.apply(get_emission_unit, axis=1)
output_fugitive_emissions['description'] = activity_data.apply(get_description, axis=1)
output_fugitive_emissions['country'] = activity_data.apply(get_country, axis=1)

In [7]:
output_fugitive_emissions

,date_month_year,refigerant_type,amount_stored,unit,scope,category_details,emission_co2e,co2e_calculation_method,co2e_unit,description,country
0,2022,HFC-134a,450,kg,scope1,NaN,659100.0,sales_approach_user,kg,office,DE
1,2022,HFC-236fa,4500,kg,scope1,NaN,38462320.0,sales_approach_user,kg,office,DE
2,2022,PFC-14,300,kg,scope1,NaN,5217810.0,sales_approach_user,kg,office,DE
3,2022,HFE-245cb2,195,kg,scope1,NaN,595140.0,sales_approach_user,kg,office,DE
4,2022,HFE-356pcc3,347,kg,scope1,NaN,535248.0,sales_approach_user,kg,office,DE
5,2022,HFE 263fb2,636,kg,scope1,NaN,1818.0,sales_approach_user,kg,office,DE
6,2022,HFC-236ea,12765,kg,scope1,NaN,18859400.0,sales_approach_user,kg,office,DE
7,2022,HFC-43-10mee,11213,kg,scope1,NaN,21299850.0,sales_approach_user,kg,office,DE
8,2022,HFC-143a,32110,kg,scope1,NaN,163430400.0,sales_approach_user,kg,office,DE
9,2022,HFC-245fa,18900,kg,scope1,NaN,18028296.0,sales_approach_user,kg,office,DE
